In [ ]:
# Just bring in some extra libraries

import numpy as np
import matplotlib.pylab as plt

%matplotlib notebook

In [ ]:
# Here is where the data lives
data_directory = "./ForMB"

In [ ]:
# Get the labels
labelfilename = "%s/ja_labels.csv" % (data_directory)
labelfile = open(labelfilename,"r")

labels = labelfile.readline()

labels = labels.split(',')

#'''
print(labels)

for i in range(0,len(labels)):
    print("%2d %s " % (i,labels[i]))
#''';

In [ ]:
# Get the start and stop parts
stridefilename = "%s/strides_smooth.csv" % (data_directory)
stridevals = np.loadtxt(stridefilename,skiprows=1,unpack=True,dtype=bytes,delimiter=',')

filenames = stridevals[0].astype(str)

# Subtract 1 from these because the values in the file assume that the frames start at 1, not 0
firstdigitized = stridevals[1].astype(int) - 1
rstart = stridevals[2].astype(int) - 1
rend = stridevals[3].astype(int) - 1

In [ ]:
# Get the start and stop parts
roughstridefilename = "%s/strides_rough.csv" % (data_directory)
roughstridevals = np.loadtxt(roughstridefilename,skiprows=1,unpack=True,dtype=bytes,delimiter=',')

roughfilenames = roughstridevals[0].astype(str)

#print(roughstridevals[1].astype(int))


# Subtract 1 from these because the values in the file assume that the frames start at 1, not 0
roughfirstdigitized = roughstridevals[1].astype(int) - 1
rstartrough = roughstridevals[2].astype(int) - 1
rendrough = roughstridevals[3].astype(int) - 1

In [ ]:
uniquefilenames = np.unique(filenames)
#print(filenames)
print(uniquefilenames)

In [ ]:
# Make a master dictionary with all the data

alldata = {}
for i,label in enumerate(labels):
    print(label)
    alldata[label] = {}
    
    # Do the smooth data
    for ufilename in uniquefilenames:
        
        jafilename = "%s/JointAnglesFiles/%s" % (data_directory,ufilename)
        javals = np.loadtxt(jafilename,skiprows=0,unpack=True,dtype=bytes,delimiter=',')

        data = {}
        for key,val in zip(labels,javals):
            data[key] = val.astype(float)

        
        index = filenames==ufilename

        # Loop over the strides
        istride = 0
        for offset,start,end in zip(firstdigitized[index],rstart[index],rend[index]):

            header = "%s:%d:%s" % (ufilename,istride,"smooth")
            alldata[label][header] = []

            s = start
            e = end
            x = data[label][s:e]

            alldata[label][header] = x
            
            istride += 1

    # Now do the rough data
    for ufilename in uniquefilenames:
        
        jafilename = "%s/JointAnglesFiles/%s" % (data_directory,ufilename)
        javals = np.loadtxt(jafilename,skiprows=0,unpack=True,dtype=bytes,delimiter=',')

        data = {}
        for key,val in zip(labels,javals):
            data[key] = val.astype(float)


        index = roughfilenames==ufilename

        istride = 0
        for offset,start,end in zip(roughfirstdigitized[index],rstartrough[index],rendrough[index]):

            header = "%s:%d:%s" % (ufilename,istride,"rough")
            alldata[label][header] = []

            s = start
            e = end
            x = data[label][s:e]

            alldata[label][header] = x
            
            istride += 1


 

In [ ]:
# Write it out to files

for label in labels:
    print(label)
    keys = alldata[label].keys()
    maxval = -1
    nkeys = len(keys)
    
    # Find the person/trial/stride with the most frames
    for key in keys:
        data = alldata[label][key]
        nvals = len(data)
        if nvals>maxval:
            maxval = nvals

    # Write it out
    output = ",".join(keys)
    output += "\n"
    for i in range(0,maxval):
        for j,key in enumerate(keys):
            nvalskey = len(alldata[label][key])
            if i>=nvalskey:
                output += " " % ()
            else:
                output += "%s" % (alldata[label][key][i])
            
            if j<nkeys-1:
                output += ","
        
        output += "\n"
    outfilename = "variable_files/%s.csv" % (label)
    outfile = open(outfilename,"w")
    outfile.write(output)
    outfile.close()
    #print(output)

In [ ]:
plt.figure()

data = alldata['RArmAdd']

keys = data.keys()
for key in keys:
    
    y = data[key]
    
    if "smooth" in key and '5_01' in key:
        #plt.plot(y,'.',alpha=0.05,color='r')
        plt.plot(y,'.',alpha=0.5,label=key)

    #elif "rough" in key:
        #plt.plot(y,'o',alpha=0.2,color='k')
    
plt.legend()
#plt.xlabel('Person',fontsize=18)
#plt.ylabel('Right elbow angle (degrees)',fontsize=18)
#plt.ylim(0,180)

    


In [ ]:
vals = np.loadtxt('./ForMB/JointAnglesFiles/5_01_xyzpts_ja.txt',unpack=True,delimiter=',',dtype=bytes)
y = vals[13].astype(float)

plt.figure()
plt.plot(y)